In [201]:
import numpy as np
import pandas as pd

In [202]:
# from sklearn.datasets import load_breast_cancer

# breast_cancer = load_breast_cancer()
# X_train = breast_cancer.data
# y_train = breast_cancer.target

In [203]:
X_train = np.asarray([[1, 2],
                      [3, 4],
                      [5, 6],
                      [7, 8],
                      [9, 10]])
y_train = np.asarray([[1],
                     [0],
                     [1],
                     [0],
                     [1]])

In [178]:
# print(breast_cancer.data)

In [204]:
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

# Fungsi sigmoid untuk numpy array
sigmoid_v = np.vectorize(sigmoid)

In [205]:
class Dense:
    def __init__(self, layers, activation):
        # self.layers = np.zeros((1, layers))
        # self.layers = np.random.randn(layers[l], layers[l-1])*0.01
        self.layers = layers
        self.activation = activation

In [206]:
class Sequential:
    def __init__(self, input_shape, denses):
        self.denses = denses
        self.input_shape = input_shape
        self.weights = []
    
    def summary():
        None
    
    def set_weight(self):
        last_dense = 0
        for j, dense in enumerate(self.denses):
            if (j == 0):
                # print("Epoch: ", epoch, dense.layers)
                self.weights.append(np.random.randn(self.input_shape, dense.layers))
                self.weights.append(np.random.rand(1, dense.layers))
                last_dense = dense.layers
            else:
                # print("Epoch: ", epoch, last_dense, dense.layers)
                self.weights.append(np.random.randn(last_dense, dense.layers))
                self.weights.append(np.random.rand(1, dense.layers))
                last_dense = dense.layers
    
    def get_weight(self):
        # print(self.weights)
        return self.weights
    
    def compile():
        None
    
    def feedForward(self, X_train, y_train, mat):
        result = X_train
        it = iter(self.weights)
        for w in it:
            result = np.matmul(result, w) + next(it)
            # print(result)
            mat.append(result)

        return sigmoid_v(result)
    
    
    def backPropagation(self, X_train, y_train, target, mat):
        # print("mat", mat)
        err = []
        for i, m in enumerate(reversed(mat)):
            if i == 0:
                # print("err: ", m * (1 - m) * (target - m))
                # print("m: ", m)
                err.insert(0, m * (1 - m) * (target - m))
            else:
                # print("err: ", m * (1 - m) * self.get_weight()[len(m) - i - 1] * err[0])
                # print("m: ", m)
                err.insert(0, m * (1 - m) * err[len(err) - 1])
        return err
    
    def updateWeights(self, X_train, y_train, mat, err, momentum, learning_rate):
        print(mat)
        print(err)
        if (momentum):
            None
        else:
            it = iter(self.weights)
            for i, w in enumerate(it):
                print(i, " a", w)
                print(i, " b", w + (learning_rate * np.multiply(err[i], mat[i])))
                self.weights[i] = w + (learning_rate * np.multiply(err[i], mat[i]))
                next(it)
    
    def fit(self, X_train, y_train, epochs):
        mat = []
        self.set_weight()
        for epoch in range(epochs):
            self.feedForward(X_train, y_train, mat)
            err = self.backPropagation(X_train, y_train, 0, mat)
            self.updateWeights(X_train, y_train, mat, err, momentum=False, learning_rate=0.2)
        return mat

In [207]:
model = Sequential(2, [
    Dense(5, 'sigmoid'),
    Dense(3, 'sigmoid'),
    Dense(1, 'sigmoid')
])

model.fit(X_train[0], y_train, 1)

[array([[-0.5405364 ,  0.55778436, -0.3398584 , -1.46500974,  1.52876232]]), array([[-0.61469796,  5.25864716, -1.68846678]]), array([[2.18421087]])]
[array([[ -4.70451745,   1.39353732,  -2.57261675, -20.40221521,
         -4.56686994]]), array([[  -5.60752526, -126.52136385,  -25.64574788]]), array([[5.64960615]])]
0  a [[-0.30327632  0.27427034 -1.14357335  1.49936386 -1.01216814]
 [-0.38879902 -0.12308817  0.24133075 -1.74868266  0.8229837 ]]
0  b [[ 0.20531627  0.42972901 -0.96870826  7.47725266 -2.40849988]
 [ 0.11979357  0.03237049  0.41619583  4.22920614 -0.57334803]]
1  a [[ 0.05368582  0.6938255  -0.42204348]
 [ 0.71806544  1.69542152 -0.28852912]
 [ 1.2186138  -0.26396734  0.69243018]
 [ 0.56369818 -1.11477283  0.1595962 ]
 [-0.19267421  1.30186705 -1.32135198]]
1  b [[   0.74307268 -132.37241671    8.23835519]
 [   1.40745231 -131.3708207     8.37186955]
 [   1.90800067 -133.33020955    9.35282885]
 [   1.25308505 -134.18101504    8.81999487]
 [   0.49671266 -131.76437517  

[array([[-0.5405364 ,  0.55778436, -0.3398584 , -1.46500974,  1.52876232]]),
 array([[-0.61469796,  5.25864716, -1.68846678]]),
 array([[2.18421087]])]